In [ ]:
!pip install antigranular

In [ ]:
import antigranular as ag
session = ag.login("GzbhmNlZC7SNhSGBf0eqm3QAFiWuNjTx", "/N4VF5ZM2M0XagOBMCv73ObdUXQSxJwgOx+B3WMpDPX0QDYeuJfHaSt8HhGWB0Jw", competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 080963de-de62-410c-b65a-7e76d291e219, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

In [ ]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+



In [ ]:
%%ag
y_train.info()

+----+-----------+-------------+---------------+---------+----------+
|    | Column    | numerical   | categorical   | dtype   | bounds   |
|----+-----------+-------------+---------------+---------+----------|
|  0 | condition | True        | False         | int64   | (0, 1)   |
+----+-----------+-------------+---------------+---------+----------+



In [ ]:
%%ag
# We can start by exploring the data, carefully using our epsilon
describe = x_train.describe(eps=0.1)
ag_print(describe)
# We can start by exploring the data, carefully using our epsilon
describe = y_train.describe(eps=0.1)
ag_print(describe)


               age          sex  ...           bs          phr
count  8104.000000  8065.000000  ...  7174.000000  7485.000000
mean     52.909313     0.709217  ...   110.838594   132.011207
std      12.498431     0.317211  ...    25.973399    43.379231
min      21.000000     0.000000  ...    67.000000    62.000000
25%      48.716715     0.066715  ...    97.572752   128.777845
50%      66.810989     0.882302  ...   113.261225   154.288070
75%      54.939593     0.792173  ...   114.189721   166.984560
max      72.218472     0.921722  ...   137.014758   205.072596

[8 rows x 6 columns]

         condition
count  7900.000000
mean      0.545388
std       0.496187
min       0.000000
25%       0.000004
50%       0.684850
75%       0.968766
max       0.994467



In [ ]:
%%ag
import tensorflow as tf
from tensorflow import keras
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



In [ ]:
%%ag
m = keras.Sequential([
                          keras.layers.Dense(256, activation='relu',input_shape=(6,)),Dropout(0.20),
                          keras.layers.Dense(128, activation='relu'),Dropout(0.20),
                          keras.layers.Dense(64, activation='relu'),Dropout(0.20),



                          keras.layers.Dense(1, activation='sigmoid')

])

dp_model = PrivateKerasModel(model=m, l2_norm_clip=1, noise_multiplier=1.1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
%%ag
dp_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=.1)
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=2)

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1e-5)

Epoch 1/20

4000/4000 - 77s - loss: 0.5576 - accuracy: 0.7065 - 77s/epoch - 19ms/step

Epoch 2/20

4000/4000 - 73s - loss: 0.4801 - accuracy: 0.7578 - 73s/epoch - 18ms/step

Epoch 3/20

4000/4000 - 72s - loss: 0.4188 - accuracy: 0.7902 - 72s/epoch - 18ms/step

Epoch 4/20

4000/4000 - 70s - loss: 0.3750 - accuracy: 0.8142 - 70s/epoch - 18ms/step

Epoch 5/20

4000/4000 - 69s - loss: 0.3472 - accuracy: 0.8312 - 69s/epoch - 17ms/step

Epoch 6/20

4000/4000 - 72s - loss: 0.3264 - accuracy: 0.8381 - 72s/epoch - 18ms/step

Epoch 7/20

4000/4000 - 76s - loss: 0.2993 - accuracy: 0.8559 - 76s/epoch - 19ms/step

Epoch 8/20

4000/4000 - 73s - loss: 0.2967 - accuracy: 0.8553 - 73s/epoch - 18ms/step

Epoch 9/20



In [ ]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=.1)
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 0s 6ms/step



Kernel interrupted successfully


In [ ]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.7225500367545452, 'logs': {'BIN_ACC': 0.7274863673661459, 'LIN_EPS': -0.004936330611600733}}



In [ ]:
%%ag
from op_diffprivlib.models import LogisticRegression


In [ ]:
%%ag
# We can generate a quick regression using epsilon
reg = LogisticRegression(epsilon=0.001, data_norm=3)

# Fit it with 3 of the SNP predictors
reg.fit(x_train[["age", "sex", "bp", "ch", "bs", "phr"]], y_train)

/code/dependencies/ag_wrapper/ag_wrapper/wrapper.py:129: UserWarning: Bounds estimation epsilon is set to 0.1. It wont be charged until a private dataframe / series ispassed to predict or transform methods.
  warnings.warn(f"Bounds estimation epsilon is set to {bounds_estimation_eps}. It wont be charged until "
/usr/local/lib/python3.10/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

